In [20]:
import xmltodict
import pandas as pd
import numpy as np
import json
import os

In [26]:
os.getcwd()
os.chdir('D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data')

In [27]:
boletas = 'D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\boletas_mayo_oficial.json'
facturas = 'D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\factura_mayo_2023_oficial.json'

In [28]:
f = open(boletas , "rb" )
jsonObjectFActuas = json.load(f)


In [31]:
data = []
for item in jsonObjectFActuas:
    jsonload = json.loads(item['bol_response'])
    dictionary = xmltodict.parse(jsonload['xml'])
    total = str(dictionary['Invoice']['cac:InvoiceLine']['cac:PricingReference']['cac:AlternativeConditionPrice']['cbc:PriceAmount']['#text'])
    total = int(total)
    gravado = round(total / (1 + 0.18), 2)
    igv = round(total - sutotal, 2)
    sutotal = total - igv

    num_correlativo = item['bol_correlativo']
    serie = item['bol_serie']
    dni = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyIdentification']['cbc:ID']['#text']
    tipo_doc = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyIdentification']['cbc:ID']['@schemeID']

    descripcion_producto = dictionary['Invoice']['cac:InvoiceLine']['cac:Item']['cbc:Description']
    codigo_producto =  dictionary['Invoice']['cac:InvoiceLine']['cac:Item']['cac:SellersItemIdentification']['cbc:ID']

    leyenda = dictionary['Invoice']['cbc:Note']['#text']
    nombres = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyLegalEntity']['cbc:RegistrationName']
    fecha_emision =  dictionary['Invoice']['cbc:IssueDate']

    description_sunat = ''
    if jsonload['sunatResponse'].get('cdrResponse', None) is not None:
        description_sunat = jsonload['sunatResponse']['cdrResponse']['description']
    elif jsonload['sunatResponse'].get('error'):
        description_sunat = jsonload['sunatResponse']['error']['message']

    COMPANY_RUC='20545280605'
    COMPANY_RZSOCIAL='ORGANIZACION EDUCATIVA AMERICANA S.A.C.'
    COMAPANY_N_COMERCIAL='OEA Educación'
    COMAPANY_ADDRESS='Av. la Marina 4to Piso Nro. 2529, San Miguel, Lima - Perú'
    COMAPANY_DEPARTAMENTO='OEA LIMA'
    COMAPANY_PROVINCIA='OEA LIMA'
    COMAPANY_DISTRITO='OEA SAN MIGUEL'
    COMAPANY_UBIGEO='150101'
    COD_TIP_NIF_EMIS='6'
    CURRENCY_MONEY='PEN'
    token = ''

    request_body = {
        "TOKEN": token,
        "items": [
            {
                "COD_ITEM": codigo_producto,
                "MNT_BRUTO": gravado,
                "MNT_PV_ITEM": gravado,
                "MNT_IGV_ITEM": igv,
                "POR_IGV_ITEM": 18,
                "VAL_VTA_ITEM": gravado,
                "COD_UNID_ITEM": "NIU",
                "TXT_DESC_ITEM": descripcion_producto,
                "VAL_UNIT_ITEM": gravado,
                "CANT_UNID_ITEM": 1,
                "COD_TIP_PRC_VTA": "01",
                "COD_TRIB_IGV_ITEM": "1000",
                "PRC_VTA_UNIT_ITEM": total,
                "COD_TIP_AFECT_IGV_ITEM": 10
            }
        ],
        "COD_MND": CURRENCY_MONEY,
        "MNT_TOT": total,
        "FEC_EMIS": fecha_emision,
        "COD_TIP_CPE": "03",
        "RETORNA_PDF": "true",
        "COD_UBI_EMIS": COMAPANY_UBIGEO,
        "NUM_NIF_EMIS": COMPANY_RUC,
        "NUM_NIF_RECP": dni,
        "TXT_VERS_UBL": "2.1",
        "COD_FORM_IMPR": "001",
        "NUM_CORRE_CPE":'',
        "NUM_SERIE_CPE":'',
        "COD_ANEXO_EMIS": "0000",
        "COD_PRCD_CARGA": "001",
        "ENVIAR_A_SUNAT": "true",
        "NOM_COMER_EMIS": COMAPANY_N_COMERCIAL,
        "MNT_TOT_GRAVADO": gravado,
        "RETORNA_XML_CDR": "true",
        "COD_TIP_NIF_EMIS": COD_TIP_NIF_EMIS,
        "COD_TIP_NIF_RECP": tipo_doc,
        "MNT_TOT_TRIB_IGV": igv,
        "NOM_RZN_SOC_EMIS": COMPANY_RZSOCIAL,
        "NOM_RZN_SOC_RECP": nombres,
        "COD_TIP_OPE_SUNAT": "0101",
        "RETORNA_XML_ENVIO": "true",
        "TXT_DMCL_FISC_EMIS": COMAPANY_ADDRESS,
        "TXT_DMCL_FISC_RECEP": 'sin descripcion',
        "TXT_VERS_ESTRUCT_UBL": "2.0"
    }


    obj = {
        'serie': serie,
        'correlativo': num_correlativo,
        'fecha_emision': fecha_emision,
        'cliente': nombres,
        'dni': dni,
        'tipo_doc': item['tipo_doc'],
        'total': total,
        'subtotal': sutotal,
        'igv': igv,
        'type_sales': item['type_sales'],
        'diplomado_name': item['courses_name'],
        'centro_costo': item['centro_costo'],
        'sunat_description': description_sunat
    }
    data.append(obj)



In [30]:
df = pd.DataFrame(data)
df.head()

,serie,correlativo,fecha_emision,cliente,dni,tipo_doc,total,subtotal,igv,type_sales,diplomado_name,centro_costo,sunat_description
0,B001,12304,2023-05-01,KAREN DEL PILAR PAREDES ROJAS,47681264,1,180,152.54,27.46,mensualidad,Diplomado en Obstetricia: Auditoría en Salud e...,302234,"La Boleta de Venta numero B001-12304, ha sido ..."
1,B001,12305,2023-05-01,MEDALITH NAOMI SAMANIEGO FLORES,75508669,1,150,127.12,22.88,mensualidad,Diplomado en Enfermería: Atención de Enfermerí...,None,"La Boleta de Venta numero B001-12305, ha sido ..."
2,B001,12306,2023-05-01,JOSHELIN RAMIREZ VERA,75270716,1,220,186.44,33.56,mensualidad,Diplomado en Enfermería: Salud Ocupacional en ...,302229,"La Boleta de Venta numero B001-12306, ha sido ..."
3,B001,12307,2023-05-01,JENNY ASHLY GABRIELA MEJIA SANCHEZ,47771911,1,150,127.12,22.88,mensualidad,Diplomado en Enfermería: Enfermería en Emergen...,None,"La Boleta de Venta numero B001-12307, ha sido ..."
4,B001,12308,2023-05-01,ESTEFANY ASHLIN TENORIO CABEZAS,47594832,1,150,127.12,22.88,mensualidad,Diplomado en Enfermería: Auditoría en Salud en...,None,"La Boleta de Venta numero B001-12308, ha sido ..."


In [7]:
df.shape

(1635, 13)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1635 entries, 0 to 1634
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   serie              1635 non-null   object 
 1   correlativo        1635 non-null   int64  
 2   fecha_emision      1635 non-null   object 
 3   cliente            1635 non-null   object 
 4   dni                1635 non-null   object 
 5   tipo_doc           1635 non-null   object 
 6   total              1635 non-null   int64  
 7   subtotal           1635 non-null   float64
 8   igv                1635 non-null   float64
 9   type_sales         1635 non-null   object 
 10  diplomado_name     1635 non-null   object 
 11  centro_costo       345 non-null    object 
 12  sunat_description  1635 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 166.2+ KB


In [17]:
df.to_excel('D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\factura_mes_mayo_2023v1.xlsx')

In [19]:
data =  [2659.12,2659.12, 2659.12, 2659.12, 2659.12, 2659.12, 2659.12, 2659.12]
data = round(sum(data), 2)
print(data)

21272.96
